In [0]:
print("Healthy")

In [0]:
from re import search
from sqlalchemy import create_engine

import pandas as pd 
import numpy as np 

import mysql.connector

import warnings
warnings.filterwarnings('ignore')

In [0]:
from datetime import datetime, timedelta
yesterday =  (datetime.today() - timedelta(days = 1)).strftime('%Y-%m-%d')
yesterday = 'Jun-05 to Jun-20'
yesterday_format2 =  (datetime.today() - timedelta(days = 1)).strftime('%b_%d')
print(yesterday)
print(yesterday_format2)

In [0]:
from re import search
from sqlalchemy import create_engine
import duckdb

import pandas as pd 
import numpy as np 
import mysql.connector


import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)


import json
with open('/Workspace/Credentials/db_data.json', 'r') as fp:
    data = json.load(fp)


Mifosdb = mysql.connector.connect(
  host=data['mifos']['host'],
  user=data['mifos']['user'],
  passwd=data['mifos']['passwd'],
  database = data['mifos']['database']
)


host = data['redshift']['host']
user = data['redshift']['user']
passwd = data['redshift']['passwd']
database = data['redshift']['database']

conn = create_engine(f"postgresql+psycopg2://{user}:{passwd}@{host}:5439/{database}")
pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [0]:
username = data['auto-report']['email']
passwd = data['auto-report']['passwd']
print(username)

In [0]:
staff = pd.read_excel("/Workspace/ReportDump/Staff_List/Feb 24 Staff List.xlsx")
staff.head()

In [0]:
sql = f'''
SELECT
sa.account_no,
mc.display_name AS staff_name,
mc.id AS staff_client_id,
mc.referral_id
FROM m_savings_account sa
JOIN m_client mc ON mc.id = sa.client_id
WHERE sa.account_no IN ({','.join(str(x) for x in staff['Account Numbers'])})
AND mc.referral_id IS NOT NULL
GROUP BY mc.id

'''
staff = pd.read_sql_query(sql, Mifosdb)

other_staff = {
    'account_no': [''],
    'staff_name': [''],
    'staff_client_id': ['others'],
    'referral_id': [''],
}

other_staff = pd.DataFrame(other_staff)
staff = staff.append(other_staff, ignore_index=True)

staff.head(2)

In [0]:
display(staff)

In [0]:
from pyspark.sql.functions import lit
import dask
import dask.dataframe as dd
from sqlalchemy import create_engine

# MIFOS_DB
mifos_driver = "org.mariadb.jdbc.Driver"
mifos_database_host = data['mifos']['host']
mifos_database_port = "3306" # update if you use a non-default port
mifos_database_name = data['mifos']['database']
mifos_user=data['mifos']['user']
mifos_password = data['mifos']['passwd']
mifos_url = f"jdbc:mysql://{mifos_database_host}:{mifos_database_port}/{mifos_database_name}"

# Define function to extract data from dbs
def extract_function(query, url, user, password):
    df = spark.read \
                .format("jdbc") \
                .option("query", query) \
                .option("url", url) \
                .option("user", user) \
                .option("password", password) \
                .option("connectTimeout", "1000") \
                .option("treatEmptyValueAsNulls","true") \
                .option("maxRowsInMemory",200) \
                .load()
    return df


## For Mifos
sql = f'''
SELECT
mc.referred_by_id AS staff_client_id,
IFNULL(SUM(CASE WHEN left(st.transaction_date,10) = '2024-06-05' THEN st.amount END),0) AS Jun_05_total_inflow,
IFNULL(SUM(CASE WHEN left(st.transaction_date,10) = '2024-06-06' THEN st.amount END),0) AS Jun_06_total_inflow,
IFNULL(SUM(CASE WHEN left(st.transaction_date,10) = '2024-06-07' THEN st.amount END),0) AS Jun_07_total_inflow,
IFNULL(SUM(CASE WHEN left(st.transaction_date,10) = '2024-06-08' THEN st.amount END),0) AS Jun_08_total_inflow,
IFNULL(SUM(CASE WHEN left(st.transaction_date,10) = '2024-06-09' THEN st.amount END),0) AS Jun_09_total_inflow,
IFNULL(SUM(CASE WHEN left(st.transaction_date,10) = '2024-06-10' THEN st.amount END),0) AS Jun_10_total_inflow,
IFNULL(SUM(CASE WHEN left(st.transaction_date,10) = '2024-06-11' THEN st.amount END),0) AS Jun_11_total_inflow,
IFNULL(SUM(CASE WHEN left(st.transaction_date,10) = '2024-06-12' THEN st.amount END),0) AS Jun_12_total_inflow,
IFNULL(SUM(CASE WHEN left(st.transaction_date,10) = '2024-06-13' THEN st.amount END),0) AS Jun_13_total_inflow,
IFNULL(SUM(CASE WHEN left(st.transaction_date,10) = '2024-06-14' THEN st.amount END),0) AS Jun_14_total_inflow,
IFNULL(SUM(CASE WHEN left(st.transaction_date,10) = '2024-06-15' THEN st.amount END),0) AS Jun_15_total_inflow,
IFNULL(SUM(CASE WHEN left(st.transaction_date,10) = '2024-06-16' THEN st.amount END),0) AS Jun_16_total_inflow,
IFNULL(SUM(CASE WHEN left(st.transaction_date,10) = '2024-06-17' THEN st.amount END),0) AS Jun_17_total_inflow,
IFNULL(SUM(CASE WHEN left(st.transaction_date,10) = '2024-06-18' THEN st.amount END),0) AS Jun_18_total_inflow,
IFNULL(SUM(CASE WHEN left(st.transaction_date,10) = '2024-06-19' THEN st.amount END),0) AS Jun_19_total_inflow,
IFNULL(SUM(CASE WHEN left(st.transaction_date,10) = '2024-06-20' THEN st.amount END),0) AS Jun_20_total_inflow,

IFNULL(SUM(CASE WHEN sa.product_id = '36' and left(st.transaction_date,10) = '2024-06-05' then st.amount END),0) AS Jun_05_FD_inflow,
IFNULL(SUM(CASE WHEN sa.product_id = '36' and left(st.transaction_date,10) = '2024-06-06' then st.amount END),0) AS Jun_06_FD_inflow,
IFNULL(SUM(CASE WHEN sa.product_id = '36' and left(st.transaction_date,10) = '2024-06-07' then st.amount END),0) AS Jun_07_FD_inflow,
IFNULL(SUM(CASE WHEN sa.product_id = '36' and left(st.transaction_date,10) = '2024-06-08' then st.amount END),0) AS Jun_08_FD_inflow,
IFNULL(SUM(CASE WHEN sa.product_id = '36' and left(st.transaction_date,10) = '2024-06-09' then st.amount END),0) AS Jun_09_FD_inflow,
IFNULL(SUM(CASE WHEN sa.product_id = '36' and left(st.transaction_date,10) = '2024-06-10' then st.amount END),0) AS Jun_10_FD_inflow,
IFNULL(SUM(CASE WHEN sa.product_id = '36' and left(st.transaction_date,10) = '2024-06-11' then st.amount END),0) AS Jun_11_FD_inflow,
IFNULL(SUM(CASE WHEN sa.product_id = '36' and left(st.transaction_date,10) = '2024-06-12' then st.amount END),0) AS Jun_12_FD_inflow,
IFNULL(SUM(CASE WHEN sa.product_id = '36' and left(st.transaction_date,10) = '2024-06-13' then st.amount END),0) AS Jun_13_FD_inflow,
IFNULL(SUM(CASE WHEN sa.product_id = '36' and left(st.transaction_date,10) = '2024-06-14' then st.amount END),0) AS Jun_14_FD_inflow,
IFNULL(SUM(CASE WHEN sa.product_id = '36' and left(st.transaction_date,10) = '2024-06-15' then st.amount END),0) AS Jun_15_FD_inflow,
IFNULL(SUM(CASE WHEN sa.product_id = '36' and left(st.transaction_date,10) = '2024-06-16' then st.amount END),0) AS Jun_16_FD_inflow,
IFNULL(SUM(CASE WHEN sa.product_id = '36' and left(st.transaction_date,10) = '2024-06-17' then st.amount END),0) AS Jun_17_FD_inflow,
IFNULL(SUM(CASE WHEN sa.product_id = '36' and left(st.transaction_date,10) = '2024-06-18' then st.amount END),0) AS Jun_18_FD_inflow,
IFNULL(SUM(CASE WHEN sa.product_id = '36' and left(st.transaction_date,10) = '2024-06-19' then st.amount END),0) AS Jun_19_FD_inflow,
IFNULL(SUM(CASE WHEN sa.product_id = '36' and left(st.transaction_date,10) = '2024-06-20' then st.amount END),0) AS Jun_20_FD_inflow
FROM m_savings_account sa
JOIN m_client mc ON mc.id = sa.client_id
JOIN m_savings_product sp ON sp.id = sa.product_id
LEFT JOIN m_savings_account_transaction st ON st.savings_account_id = sa.id
WHERE st.transaction_type_enum = 1
AND left(st.transaction_date,10) >= '2024-06-05'
AND left(st.transaction_date,10) <= '2024-06-20'
AND mc.referred_by_id IN ({','.join(str(x) for x in staff['staff_client_id'])})
GROUP BY mc.referred_by_id
'''       
# apply function
mifos_df = extract_function(query=sql, url=mifos_url, user=mifos_user, password=mifos_password)
mifos_df.persist()
deposit = mifos_df.toPandas()

In [0]:
from pyspark.sql.functions import lit
import dask
import dask.dataframe as dd
from sqlalchemy import create_engine

# MIFOS_DB
mifos_driver = "org.mariadb.jdbc.Driver"
mifos_database_host = data['mifos']['host']
mifos_database_port = "3306" # update if you use a non-default port
mifos_database_name = data['mifos']['database']
mifos_user=data['mifos']['user']
mifos_password = data['mifos']['passwd']
mifos_url = f"jdbc:mysql://{mifos_database_host}:{mifos_database_port}/{mifos_database_name}"

# Define function to extract data from dbs
def extract_function(query, url, user, password):
    df = spark.read \
                .format("jdbc") \
                .option("query", query) \
                .option("url", url) \
                .option("user", user) \
                .option("password", password) \
                .option("connectTimeout", "1000") \
                .option("treatEmptyValueAsNulls","true") \
                .option("maxRowsInMemory",200) \
                .load()
    return df


## For Mifos
sql = f'''
SELECT
CASE WHEN mc.referred_by_id IS NULL THEN 'others' ELSE mc.referred_by_id END AS staff_client_id,
IFNULL(SUM(CASE WHEN left(st.transaction_date,10) = '2024-06-05' THEN st.amount END),0) AS Jun_05_total_inflow,
IFNULL(SUM(CASE WHEN left(st.transaction_date,10) = '2024-06-06' THEN st.amount END),0) AS Jun_06_total_inflow,
IFNULL(SUM(CASE WHEN left(st.transaction_date,10) = '2024-06-07' THEN st.amount END),0) AS Jun_07_total_inflow,
IFNULL(SUM(CASE WHEN left(st.transaction_date,10) = '2024-06-08' THEN st.amount END),0) AS Jun_08_total_inflow,
IFNULL(SUM(CASE WHEN left(st.transaction_date,10) = '2024-06-09' THEN st.amount END),0) AS Jun_09_total_inflow,
IFNULL(SUM(CASE WHEN left(st.transaction_date,10) = '2024-06-10' THEN st.amount END),0) AS Jun_10_total_inflow,
IFNULL(SUM(CASE WHEN left(st.transaction_date,10) = '2024-06-11' THEN st.amount END),0) AS Jun_11_total_inflow,
IFNULL(SUM(CASE WHEN left(st.transaction_date,10) = '2024-06-12' THEN st.amount END),0) AS Jun_12_total_inflow,
IFNULL(SUM(CASE WHEN left(st.transaction_date,10) = '2024-06-13' THEN st.amount END),0) AS Jun_13_total_inflow,
IFNULL(SUM(CASE WHEN left(st.transaction_date,10) = '2024-06-14' THEN st.amount END),0) AS Jun_14_total_inflow,
IFNULL(SUM(CASE WHEN left(st.transaction_date,10) = '2024-06-15' THEN st.amount END),0) AS Jun_15_total_inflow,
IFNULL(SUM(CASE WHEN left(st.transaction_date,10) = '2024-06-16' THEN st.amount END),0) AS Jun_16_total_inflow,
IFNULL(SUM(CASE WHEN left(st.transaction_date,10) = '2024-06-17' THEN st.amount END),0) AS Jun_17_total_inflow,
IFNULL(SUM(CASE WHEN left(st.transaction_date,10) = '2024-06-18' THEN st.amount END),0) AS Jun_18_total_inflow,
IFNULL(SUM(CASE WHEN left(st.transaction_date,10) = '2024-06-19' THEN st.amount END),0) AS Jun_19_total_inflow,
IFNULL(SUM(CASE WHEN left(st.transaction_date,10) = '2024-06-20' THEN st.amount END),0) AS Jun_20_total_inflow,

IFNULL(SUM(CASE WHEN left(st.transaction_date,10) = '2024-06-21' THEN st.amount END),0) AS Jun_21_total_inflow,
IFNULL(SUM(CASE WHEN left(st.transaction_date,10) = '2024-06-22' THEN st.amount END),0) AS Jun_22_total_inflow,
IFNULL(SUM(CASE WHEN left(st.transaction_date,10) = '2024-06-23' THEN st.amount END),0) AS Jun_23_total_inflow,

IFNULL(SUM(CASE WHEN sa.product_id = '36' and left(st.transaction_date,10) = '2024-06-05' then st.amount END),0) AS Jun_05_FD_inflow,
IFNULL(SUM(CASE WHEN sa.product_id = '36' and left(st.transaction_date,10) = '2024-06-06' then st.amount END),0) AS Jun_06_FD_inflow,
IFNULL(SUM(CASE WHEN sa.product_id = '36' and left(st.transaction_date,10) = '2024-06-07' then st.amount END),0) AS Jun_07_FD_inflow,
IFNULL(SUM(CASE WHEN sa.product_id = '36' and left(st.transaction_date,10) = '2024-06-08' then st.amount END),0) AS Jun_08_FD_inflow,
IFNULL(SUM(CASE WHEN sa.product_id = '36' and left(st.transaction_date,10) = '2024-06-09' then st.amount END),0) AS Jun_09_FD_inflow,
IFNULL(SUM(CASE WHEN sa.product_id = '36' and left(st.transaction_date,10) = '2024-06-10' then st.amount END),0) AS Jun_10_FD_inflow,
IFNULL(SUM(CASE WHEN sa.product_id = '36' and left(st.transaction_date,10) = '2024-06-11' then st.amount END),0) AS Jun_11_FD_inflow,
IFNULL(SUM(CASE WHEN sa.product_id = '36' and left(st.transaction_date,10) = '2024-06-12' then st.amount END),0) AS Jun_12_FD_inflow,
IFNULL(SUM(CASE WHEN sa.product_id = '36' and left(st.transaction_date,10) = '2024-06-13' then st.amount END),0) AS Jun_13_FD_inflow,
IFNULL(SUM(CASE WHEN sa.product_id = '36' and left(st.transaction_date,10) = '2024-06-14' then st.amount END),0) AS Jun_14_FD_inflow,
IFNULL(SUM(CASE WHEN sa.product_id = '36' and left(st.transaction_date,10) = '2024-06-15' then st.amount END),0) AS Jun_15_FD_inflow,
IFNULL(SUM(CASE WHEN sa.product_id = '36' and left(st.transaction_date,10) = '2024-06-16' then st.amount END),0) AS Jun_16_FD_inflow,
IFNULL(SUM(CASE WHEN sa.product_id = '36' and left(st.transaction_date,10) = '2024-06-17' then st.amount END),0) AS Jun_17_FD_inflow,
IFNULL(SUM(CASE WHEN sa.product_id = '36' and left(st.transaction_date,10) = '2024-06-18' then st.amount END),0) AS Jun_18_FD_inflow,
IFNULL(SUM(CASE WHEN sa.product_id = '36' and left(st.transaction_date,10) = '2024-06-19' then st.amount END),0) AS Jun_19_FD_inflow,
IFNULL(SUM(CASE WHEN sa.product_id = '36' and left(st.transaction_date,10) = '2024-06-20' then st.amount END),0) AS Jun_20_FD_inflow,
IFNULL(SUM(CASE WHEN sa.product_id = '36' and left(st.transaction_date,10) = '2024-06-21' then st.amount END),0) AS Jun_21_FD_inflow,
IFNULL(SUM(CASE WHEN sa.product_id = '36' and left(st.transaction_date,10) = '2024-06-22' then st.amount END),0) AS Jun_22_FD_inflow,
IFNULL(SUM(CASE WHEN sa.product_id = '36' and left(st.transaction_date,10) = '2024-06-23' then st.amount END),0) AS Jun_23_FD_inflow
FROM m_savings_account sa
JOIN m_client mc ON mc.id = sa.client_id
JOIN m_savings_product sp ON sp.id = sa.product_id
LEFT JOIN m_savings_account_transaction st ON st.savings_account_id = sa.id
WHERE st.transaction_type_enum = 1
AND left(st.transaction_date,10) >= '2024-06-05'
AND left(st.transaction_date,10) <= '2024-06-23'
AND mc.referred_by_id IS NULL
GROUP BY mc.referred_by_id
'''       
# apply function
mifos_df = extract_function(query=sql, url=mifos_url, user=mifos_user, password=mifos_password)
mifos_df.persist()
deposit = mifos_df.toPandas()

In [0]:
deposit.head()

In [0]:
full_df = staff[['staff_client_id', 'staff_name','referral_id']].merge(deposit, on = 'staff_client_id', how = 'left')
#full_df = full_df.sort_values(by=['staff_client_id'], ascending=False)
full_df.head()

In [0]:
full_df.shape

In [0]:
with pd.ExcelWriter(f"/Workspace/ReportDump/Deposit_Drive/Deposit_Drive_{yesterday}.xlsx") as writer:
    full_df.to_excel(writer, sheet_name = f'{yesterday}', index = False)

In [0]:
# FILE TO SEND AND ITS PATH
filename = f'Deposit_Drive_{yesterday}.xlsx'
SourcePathName  = "/Workspace/ReportDump/Deposit_Drive/" + filename


In [0]:
import csv, smtplib, ssl
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase
from email.mime.text import MIMEText
from email import encoders

#rotimi.awofisibe@vfd-mfb.com, mercy.ejemudaro@vfd-mfb.com, 

msg = MIMEMultipart()
msg['From'] = 'auto-report@vfdtech.ng'
msg['To'] = 'Oladapo.Omolaja@vfdtech.ng'
#msg['CC'] = 'data-team@vfdtech.ng'
msg['Subject'] = f'Staff Deposit Drive {yesterday}'
body = f"""
Hello Team,

Please find attached report for {yesterday}.

Regards,
DBR
"""
msg.attach(MIMEText(body, 'plain'))



## ATTACHMENT PART OF THE CODE IS HERE
attachment = open(SourcePathName, 'rb')
part = MIMEBase('application', "octet-stream")
part.set_payload((attachment).read())
encoders.encode_base64(part)
part.add_header('Content-Disposition', "attachment; filename= %s" % filename)
msg.attach(part)

server = smtplib.SMTP('smtp.office365.com', 587)  ### put your relevant SMTP here
server.ehlo()
server.starttls()
server.ehlo()
server.login(username, passwd)  ### if applicable
server.send_message(msg)
server.quit()